In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.set_visible_devices([], 'GPU')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling1D, Input, concatenate, LSTM, Add
from tensorflow.keras.layers import SpatialDropout1D, Bidirectional, Activation, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
import numpy as np
import os
import time
from datetime import datetime
import gc

In [ ]:
# Keep memory optimizations
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.set_visible_devices([], 'GPU')
tf.config.run_functions_eagerly(True)

In [ ]:
# Set memory growth for physical devices
physical_devices = tf.config.list_physical_devices()
for device in physical_devices:
    try:
        tf.config.experimental.set_memory_growth(device, True)
    except:
        pass

In [ ]:
# Set TF memory limits explicitly
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
        )
    except:
        pass

In [ ]:
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.18.0


In [ ]:
# Constants - slightly increased to find better balance
SEQUENCE_LENGTH = 5000  # Keep same length
NUM_CLASSES = 4
CLASS_NAMES = ["Bike Trotting", "Jack Hammering", "Jumping", "Walking"]
BATCH_SIZE = 8  # Increased from 4 to 8 for better gradient estimation
MAX_TRAIN_SAMPLES = 12000  # Increased from 8000 to 12000
MAX_TEST_SAMPLES = 3000  # Increased from 2000 to 3000

In [ ]:
# Data augmentation functions - keep but reduce noise slightly for underfitting
def add_noise(signals, noise_factor=0.05):  # Reduced from 0.08 to 0.05
    """Add random noise to signal batch with scaling based on signal amplitude"""
    batch_size = signals.shape[0]
    results = signals.copy()

    for i in range(batch_size):
        # Scale noise based on signal amplitude
        signal_max = np.max(np.abs(signals[i]))
        if signal_max > 0:  # Avoid division by zero
            noise_scale = signal_max * noise_factor
            noise = np.random.normal(0, noise_scale, signals[i].shape)
            results[i] = signals[i] + noise

    return results

In [ ]:
def time_shift(signals, shift_limit=0.2):  # Reduced from 0.3 to 0.2
    """Apply random time shift to batch of signals with wrap-around"""
    batch_size = signals.shape[0]
    results = np.zeros_like(signals)

    for i in range(batch_size):
        signal = signals[i].copy()
        shift = np.random.uniform(-shift_limit, shift_limit)
        shift_samples = int(signal.shape[0] * shift)

        if shift_samples > 0:
            results[i, :-shift_samples] = signal[shift_samples:]
            results[i, -shift_samples:] = signal[:shift_samples]
        elif shift_samples < 0:
            shift_samples = abs(shift_samples)
            results[i, shift_samples:] = signal[:-shift_samples]
            results[i, :shift_samples] = signal[-shift_samples:]
        else:
            results[i] = signal

    return results

In [ ]:
def amplitude_scaling(signals, scaling_factor_range=(0.8, 1.2)):  # Tightened from (0.7,1.3) to (0.8,1.2)
    """Apply random amplitude scaling to batch with different factors for each sample"""
    batch_size = signals.shape[0]
    results = signals.copy()

    for i in range(batch_size):
        scaling_factor = np.random.uniform(*scaling_factor_range)
        results[i] = signals[i] * scaling_factor

    return results

In [ ]:
def frequency_masking(signals, max_masks=2, max_width=0.08):  # Reduced for less aggressive masking
    """Apply random frequency domain masking for increased robustness"""
    batch_size = signals.shape[0]
    seq_length = signals.shape[1]
    results = signals.copy()

    for i in range(batch_size):
        n_masks = np.random.randint(1, max_masks + 1)
        for _ in range(n_masks):
            width = int(seq_length * np.random.uniform(0.01, max_width))
            start = np.random.randint(0, seq_length - width)
            mask_value = np.mean(results[i])
            results[i, start:start+width] = mask_value

    return results

In [ ]:
# Augmentation pipeline with reduced probability for underfitting case
def augment_batch(X_batch, y_batch, augment_probability=0.5):  # Reduced from 0.7 to 0.5
    """Apply a sequence of augmentations with given probability - adjusted for underfitting"""
    batch_size = X_batch.shape[0]
    augment_mask = np.random.random(batch_size) < augment_probability

    if not np.any(augment_mask):
        return X_batch, y_batch

    X_batch_aug = X_batch.copy()
    indices = np.where(augment_mask)[0]

    # Apply only some augmentations to prevent too much distortion
    if np.random.random() < 0.6:  # Reduced from 0.7
        X_batch_aug[indices] = time_shift(X_batch_aug[indices], shift_limit=0.2)

    if np.random.random() < 0.5:  # Reduced from 0.7
        X_batch_aug[indices] = add_noise(X_batch_aug[indices], noise_factor=0.05)

    if np.random.random() < 0.5:  # Reduced from 0.6
        X_batch_aug[indices] = amplitude_scaling(X_batch_aug[indices],
                                              scaling_factor_range=(0.8, 1.2))

    # Reduce frequency masking probability for underfitting
    if np.random.random() < 0.3:  # Reduced from 0.4
        X_batch_aug[indices] = frequency_masking(X_batch_aug[indices])

    return X_batch_aug, y_batch

In [ ]:
# Improved residual block with better capacity while maintaining efficiency
def residual_block(x, filters, kernel_size=3, strides=1, dropout_rate=0.2):  # Reduced dropout from 0.25
    """Create a residual block with skip connection - balanced for capacity vs memory"""
    skip = x

    # Path A: Main path with more capacity but still efficient
    x = Conv1D(filters, kernel_size, strides=strides, padding='same',
               kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5))(x)  # Reduced regularization for underfitting
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Add a second convolution for more capacity - key for addressing underfitting
    x = Conv1D(filters, kernel_size, padding='same',
               kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5))(x)
    x = BatchNormalization()(x)

    # Path B: Skip connection
    if strides > 1 or skip.shape[-1] != filters:
        skip = Conv1D(filters, 1, strides=strides, padding='same')(skip)
        skip = BatchNormalization()(skip)

    # Combine paths
    x = Add()([x, skip])
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)

    return x

In [ ]:
# Balanced model with improved capacity for addressing underfitting
def create_balanced_hybrid_model(sequence_length):
    """Create a balanced hybrid CNN-LSTM model that addresses underfitting while staying memory-efficient"""
    # Input layer
    inputs = Input(shape=(sequence_length, 1), name="input_layer")

    # First convolutional block with more filters
    x = residual_block(inputs, 16, kernel_size=7, strides=2, dropout_rate=0.2)  # Increased from 8 to 16 filters, larger kernel
    x = MaxPooling1D(pool_size=2)(x)

    # Second convolutional block with more filters
    x = residual_block(x, 32, kernel_size=5, strides=2, dropout_rate=0.2)  # Increased from 16 to 32 filters
    x = MaxPooling1D(pool_size=2)(x)

    # Add a third convolutional block for more capacity
    x = residual_block(x, 64, kernel_size=3, strides=1, dropout_rate=0.2)  # New layer with 64 filters
    x = MaxPooling1D(pool_size=2)(x)

    # Multiple feature extraction paths
    gap_features = GlobalAveragePooling1D()(x)
    gmp_features = GlobalMaxPooling1D()(x)

    # Use bidirectional LSTM for better sequence understanding
    lstm_features = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.0))(x)  # Increased from 16 to 32

    # Combine features from all paths for better representation
    combined = concatenate([gap_features, gmp_features, lstm_features])

    # Enhanced dense layers with balanced regularization
    x = Dense(64, kernel_regularizer=l1_l2(l1=1e-6, l2=1e-5))(combined)  # Increased from 32 to 64 units
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)  # Reduced from 0.4 to address underfitting

    # Output layer
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile with Adam optimizer and slightly higher learning rate for faster convergence
    model.compile(
        optimizer=Adam(learning_rate=1e-3),  # Increased from 5e-4 to 1e-3
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
# Optimized training function with adjusted early stopping for underfitting
def train_model(model, X_train, y_train, X_val, y_val, batch_size=BATCH_SIZE, epochs=30, output_dir=None):
    """Train model with balanced regularization to prevent underfitting"""
    print(f"Training model for {epochs} epochs with batch_size={batch_size}...")

    # Define callbacks with adjusted parameters for underfitting
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=5,  # Increased from 3 to 5 to allow more training time
            restore_best_weights=True,
            min_delta=0.001
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,  # Less aggressive reduction (from 0.3 to 0.5)
            patience=3,  # Increased from 2 to 3
            min_lr=1e-6,
            verbose=1
        )
    ]

    if output_dir:
        callbacks.append(
            ModelCheckpoint(
                filepath=os.path.join(output_dir, 'best_model.h5'),
                monitor='val_loss',
                save_best_only=True,
                verbose=1
            )
        )

    # Data generator with balanced augmentation
    def generator(X, y, batch_size, augment=True):
        indices = np.arange(len(X))

        while True:
            np.random.shuffle(indices)
            for start_idx in range(0, len(indices), batch_size):
                if len(indices) - start_idx < batch_size:
                    continue

                batch_indices = indices[start_idx:start_idx + batch_size]
                X_batch = X[batch_indices].copy()
                y_batch = y[batch_indices].copy()

                if augment:
                    X_batch, y_batch = augment_batch(X_batch, y_batch, augment_probability=0.5)

                gc.collect()
                yield X_batch, y_batch

    # Calculate steps per epoch - balanced for coverage vs memory
    steps_per_epoch = min(len(X_train) // batch_size, 80)  # Increased from 50 to 80
    validation_steps = min(len(X_val) // batch_size, 40)   # Increased from 25 to 40

    try:
        # Train using fit with generator
        history = model.fit(
            generator(X_train, y_train, batch_size, augment=True),
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_data=generator(X_val, y_val, batch_size, augment=False),
            validation_steps=validation_steps,
            callbacks=callbacks,
            verbose=1,
            use_multiprocessing=False,
            workers=1,
            max_queue_size=2
        )

        return history
    except Exception as e:
        print(f"Error during training: {e}")
        gc.collect()

        print("\nRetrying with a more conservative approach...")

        retry_batch_size = max(2, batch_size // 2)  # Ensure minimum batch of 2
        retry_steps = max(20, steps_per_epoch // 2)
        retry_val_steps = max(10, validation_steps // 2)

        history = model.fit(
           generator(X_train, y_train, retry_batch_size, augment=True),
           steps_per_epoch=retry_steps,
           epochs=epochs,
           validation_data=generator(X_val, y_val, retry_batch_size, augment=False),
           validation_steps=retry_val_steps,
           callbacks=callbacks,
           verbose=1
        )

        return history

In [ ]:
# Function to downsample data efficiently
def downsample_data(X, target_length):
    """Downsample time series data to target length with memory optimizations"""
    if X.shape[1] <= target_length:
        return X

    original_length = X.shape[1]

    # Process in batches to reduce memory usage
    batch_size = 100
    downsampled_X = np.zeros((X.shape[0], target_length, X.shape[2] if len(X.shape) > 2 else 1), dtype=np.float32)

    for start_idx in range(0, X.shape[0], batch_size):
        end_idx = min(start_idx + batch_size, X.shape[0])

        # Get batch
        batch = X[start_idx:end_idx]

        # Process each sample in the batch
        for i in range(batch.shape[0]):
            # Reshape if needed
            x_sample = batch[i]
            if len(x_sample.shape) == 1:
                x_sample = x_sample.reshape(-1, 1)

            # Calculate indices for downsampling
            indices = np.linspace(0, original_length-1, target_length, dtype=int)
            downsampled_X[start_idx + i] = x_sample[indices]

        # Clean up batch memory
        del batch
        gc.collect()

    return downsampled_X

In [ ]:
# Function to load and preprocess data with memory efficiency
def load_and_preprocess_data(x_train_path, y_train_path, x_test_path, y_test_path,
                           max_train_samples=MAX_TRAIN_SAMPLES, max_test_samples=MAX_TEST_SAMPLES):
    """Load and preprocess data with balanced memory optimizations"""
    print("Loading data...")

    try:
        # Load data in chunks with explicit memory mapping
        X_train = np.load(x_train_path, mmap_mode='r')
        y_train = np.load(y_train_path)
        X_test = np.load(x_test_path, mmap_mode='r')
        y_test = np.load(y_test_path)

        # Get actual shapes before limiting
        print(f"Original shapes - X_train: {X_train.shape}, X_test: {X_test.shape}")

        # Downsampling
        target_length = SEQUENCE_LENGTH
        print(f"Downsampling data from {X_train.shape[1]} to {target_length} points...")

        # Limit data size with stratified sampling to maintain class distribution
        if max_train_samples and max_train_samples < X_train.shape[0]:
            # Stratified sampling to maintain class balance
            classes = np.argmax(y_train, axis=1) if len(y_train.shape) > 1 else y_train
            train_indices = []

            for c in range(NUM_CLASSES):
                class_indices = np.where(classes == c)[0]
                # Calculate samples per class
                samples_per_class = max(1, max_train_samples // NUM_CLASSES)
                if len(class_indices) > samples_per_class:
                    selected = np.random.choice(class_indices, samples_per_class, replace=False)
                    train_indices.extend(selected)
                else:
                    train_indices.extend(class_indices)

            train_indices = np.array(train_indices)
            np.random.shuffle(train_indices)

            # Limit to max_train_samples
            if len(train_indices) > max_train_samples:
                train_indices = train_indices[:max_train_samples]

            # Create new arrays
            X_train_limited = X_train[train_indices]
            y_train_limited = y_train[train_indices]
        else:
            # Create a copy with limited memory usage
            X_train_limited = np.array(X_train[:], dtype=np.float32)
            y_train_limited = np.array(y_train[:], dtype=np.float32)

        # Apply similar process for test data
        if max_test_samples and max_test_samples < X_test.shape[0]:
            classes = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test
            test_indices = []

            for c in range(NUM_CLASSES):
                class_indices = np.where(classes == c)[0]
                samples_per_class = max(1, max_test_samples // NUM_CLASSES)
                if len(class_indices) > samples_per_class:
                    selected = np.random.choice(class_indices, samples_per_class, replace=False)
                    test_indices.extend(selected)
                else:
                    test_indices.extend(class_indices)

            test_indices = np.array(test_indices)
            np.random.shuffle(test_indices)

            if len(test_indices) > max_test_samples:
                test_indices = test_indices[:max_test_samples]

            X_test_limited = X_test[test_indices]
            y_test_limited = y_test[test_indices]
        else:
            X_test_limited = np.array(X_test[:], dtype=np.float32)
            y_test_limited = np.array(y_test[:], dtype=np.float32)

        # Free original arrays from memory
        del X_train, X_test
        gc.collect()

        # Downsample in batches
        X_train_limited = downsample_data(X_train_limited, target_length)
        X_test_limited = downsample_data(X_test_limited, target_length)

        # Ensure correct shape (add channel dimension if needed)
        if len(X_train_limited.shape) == 2:
            X_train_limited = X_train_limited.reshape(X_train_limited.shape[0], X_train_limited.shape[1], 1)
        if len(X_test_limited.shape) == 2:
            X_test_limited = X_test_limited.reshape(X_test_limited.shape[0], X_test_limited.shape[1], 1)

        # Convert to float32 for memory efficiency if not already
        X_train_limited = X_train_limited.astype(np.float32)
        X_test_limited = X_test_limited.astype(np.float32)

        # Apply standardization - using mean/std for underfitting instead of robust
        # This gives the model more signal to work with
        for i in range(X_train_limited.shape[0]):
            mean = np.mean(X_train_limited[i])
            std = np.std(X_train_limited[i])
            if std < 1e-10:
                std = 1.0  # Avoid division by zero

            X_train_limited[i] = (X_train_limited[i] - mean) / std

        for i in range(X_test_limited.shape[0]):
            mean = np.mean(X_test_limited[i])
            std = np.std(X_test_limited[i])
            if std < 1e-10:
                std = 1.0

            X_test_limited[i] = (X_test_limited[i] - mean) / std

        print(f"Data loaded and preprocessed:")
        print(f"X_train shape: {X_train_limited.shape}, y_train shape: {y_train_limited.shape}")
        print(f"X_test shape: {X_test_limited.shape}, y_test shape: {y_test_limited.shape}")

        return X_train_limited, y_train_limited, X_test_limited, y_test_limited

    except Exception as e:
        print(f"Error loading data: {e}")
        raise

In [ ]:
# Main function for model training and evaluation
def train_improved_das_model(x_train_path, y_train_path, x_test_path, y_test_path, output_dir=None):
    """End-to-end function to train and evaluate the improved DAS model addressing underfitting"""
    # Create output directory
    if output_dir is None:
        output_dir = f'das_improved_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

    os.makedirs(output_dir, exist_ok=True)
    print(f"Results will be saved to: {output_dir}")

    try:
        # Set memory limit
        tf.config.experimental.set_memory_growth = True

        # Load and preprocess data
        X_train, y_train, X_test, y_test = load_and_preprocess_data(
            x_train_path, y_train_path, x_test_path, y_test_path
        )

        # Clear TensorFlow session and force garbage collection
        tf.keras.backend.clear_session()
        gc.collect()

        # Create validation split with stratification
        val_split = int(0.15 * len(X_train))  # Decreased validation from 0.2 to 0.15 for more training data

        # Stratified split for validation
        classes = np.argmax(y_train, axis=1) if len(y_train.shape) > 1 else y_train
        train_indices = []
        val_indices = []

        for c in range(NUM_CLASSES):
            class_indices = np.where(classes == c)[0]
            np.random.shuffle(class_indices)

            # Calculate split point for this class
            split_idx = int(0.85 * len(class_indices))  # Increased from 0.8 to 0.85

            train_indices.extend(class_indices[:split_idx])
            val_indices.extend(class_indices[split_idx:])

        # Shuffle the indices
        np.random.shuffle(train_indices)
        np.random.shuffle(val_indices)

        X_val = X_train[val_indices]
        y_val = y_train[val_indices]
        X_train_final = X_train[train_indices]
        y_train_final = y_train[train_indices]

        # Free up memory
        del X_train
        gc.collect()

        # Get sequence length from data
        sequence_length = X_train_final.shape[1]

        # Implement model ensemble approach
        num_models = 3  # Keep ensemble of 3 models
        models = []
        test_accuracies = []

        for model_idx in range(num_models):
            print(f"\nTraining model {model_idx+1}/{num_models} in ensemble")

            # Clear session before each model creation
            tf.keras.backend.clear_session()
            gc.collect()

            # Create a new model with the improved architecture
            print(f"Creating balanced hybrid CNN-LSTM model (ensemble member {model_idx+1})...")
            model = create_balanced_hybrid_model(sequence_length)

            # Only show summary for first model
            if model_idx == 0:
                model.summary()
                # Save model summary
                with open(os.path.join(output_dir, 'model_summary.txt'), 'w') as f:
                    model.summary(print_fn=lambda x: f.write(x + '\n'))

            # Train with different random seed for each model
            np.random.seed(model_idx * 42)
            history = train_model(
                model, X_train_final, y_train_final, X_val, y_val,
                batch_size=BATCH_SIZE,
                epochs=30,  # Increased from 20 to 30 for more learning
                output_dir=os.path.join(output_dir, f'model_{model_idx+1}')
            )

            # Evaluate on test set in batches to avoid memory issues
            print(f"Evaluating model {model_idx+1} on test set...")
            test_loss = 0
            test_acc = 0
            num_batches = 0

            for i in range(0, len(X_test), BATCH_SIZE):
                X_batch = X_test[i:min(i+BATCH_SIZE, len(X_test))]
                y_batch = y_test[i:min(i+BATCH_SIZE, len(X_test))]
                batch_loss, batch_acc = model.evaluate(X_batch, y_batch, verbose=0)
                test_loss += batch_loss
                test_acc += batch_acc
                num_batches += 1

            test_loss /= num_batches
            test_acc /= num_batches
            test_accuracies.append(test_acc)

            print(f"Model {model_idx+1} test accuracy: {test_acc:.4f}")

            # Save the model
            model.save(os.path.join(output_dir, f'model_{model_idx+1}.keras'))
            models.append(model)

            # Force garbage collection between models
            gc.collect()

        # Calculate ensemble statistics
        avg_acc = np.mean(test_accuracies)
        max_acc = np.max(test_accuracies)

        print(f"\nEnsemble results:")
        print(f"Average test accuracy: {avg_acc:.4f}")
        print(f"Best model accuracy: {max_acc:.4f}")

        # Save ensemble results
        with open(os.path.join(output_dir, 'ensemble_results.txt'), 'w') as f:
            f.write(f"Number of models in ensemble: {num_models}\n")
            for i, acc in enumerate(test_accuracies):
                f.write(f"Model {i+1} test accuracy: {acc:.4f}\n")
            f.write(f"Average test accuracy: {avg_acc:.4f}\n")
            f.write(f"Best model accuracy: {max_acc:.4f}\n")

        return models, history, avg_acc

    except Exception as e:
        print(f"Error during training: {e}")

        # Additional debugging information
        print("\nDebug Information:")
        print(f"TensorFlow version: {tf.__version__}")

        # Check memory usage
        try:
            import psutil
            process = psutil.Process(os.getpid())
            print(f"Memory usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")
        except:
            pass

        # Check if GPU is available
        gpus = tf.config.list_physical_devices('GPU')
        print(f"Available GPUs: {gpus}")

        # Suggest fallback options
        print("\nSuggested solutions:")
        print("1. Further reduce batch size and model complexity")
        print("2. Further reduce data samples")
        print("3. Further reduce sequence length")
        print("4. Run on a machine with more memory")

        raise

In [ ]:
# Example usage
if __name__ == "__main__":
    # Define paths to data files
    X_TRAIN_PATH = '/content/drive/MyDrive/X_train.npy'
    Y_TRAIN_PATH = '/content/drive/MyDrive/y_train.npy'
    X_TEST_PATH = '/content/drive/MyDrive/X_test.npy'
    Y_TEST_PATH = '/content/drive/MyDrive/y_test.npy'

    # Create output directory with timestamp
    output_dir = f'/content/drive/MyDrive/das_model_output_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

    # Print start time
    start_time = time.time()
    print(f"Starting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Train the model
    models, history, accuracy = train_improved_das_model(
        X_TRAIN_PATH, Y_TRAIN_PATH, X_TEST_PATH, Y_TEST_PATH,
        output_dir=output_dir
    )

    # Print completion time and total runtime
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Training completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total training time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print(f"Final ensemble accuracy: {accuracy:.4f}")
    print(f"Results saved to: {output_dir}")

Starting training at 2025-03-04 05:28:24
Results will be saved to: das_model_output_20250304_052824
Loading data...
Original shapes - X_train: (64000, 25000), X_test: (16000, 25000)
Downsampling data from 25000 to 5000 points...
Data loaded and preprocessed:
X_train shape: (12000, 5000, 1), y_train shape: (12000, 4)
X_test shape: (3000, 5000, 1), y_test shape: (3000, 4)

Training model 1/3 in ensemble
Creating balanced hybrid CNN-LSTM model (ensemble member 1)...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 5000, 1)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 2500, 16)       │            128 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 2500, 16)       │             64 │ conv1d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 2500, 16)       │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 2500, 16)       │          1,808 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 2500, 16)       │             32 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 2500, 16)       │             64 │ conv1d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 2500, 16)       │             64 │ conv1d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 2500, 16)       │              0 │ batch_normalization_1… │
│                           │                        │                │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 2500, 16)       │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spatial_dropout1d         │ (None, 2500, 16)       │              0 │ activation_1[0][0]     │
│ (SpatialDropout1D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 1250, 16)       │              0 │ spatial_dropout1d[0][… │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 625, 32)        │          2,592 │ max_pooling1d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 625, 32)        │            128 │ conv1d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 625, 32)        │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 625, 32)        │          5,152 │ activation_2[0][0]     │
├──────────────────────

 Total params: 69,972 (273.33 KB)

 Trainable params: 69,172 (270.20 KB)

...

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2468 - loss: 1.8261
Epoch 1: val_loss improved from inf to 1.41178, saving model to das_model_output_20250304_052824/model_1/best_model.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 112s 3s/step - accuracy: 0.2471 - loss: 1.8268 - val_accuracy: 0.2375 - val_loss: 1.4118 - learning_rate: 0.0010
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2273 - loss: 1.7547
Epoch 2: val_loss did not improve from 1.41178
40/40 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step - accuracy: 0.2283 - loss: 1.7530 - val_accuracy: 0.3000 - val_loss: 1.4310 - learning_rate: 0.0010
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2023 - loss: 1.8009
Epoch 3: val_loss did not improve from 1.41178
40/40 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.2027 - loss: 1.8005 - val_accuracy: 0.2375 - val_loss: 1.4495 - learning_rate: 0.0010
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2403 - loss: 1.5356
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 4: val_loss did not improve from 1.41178
40/40 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.2411 - loss: 1.5368 - val_accuracy: 0.3000 - val_loss: 1.43

40/40 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.3259 - loss: 1.5233 - val_accuracy: 0.3750 - val_loss: 1.3534 - learning_rate: 5.0000e-04
Epoch 6/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2848 - loss: 1.5119
Epoch 6: val_loss improved from 1.35336 to 1.28798, saving model to das_model_output_20250304_052824/model_1/best_model.h5


40/40 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.2848 - loss: 1.5127 - val_accuracy: 0.3625 - val_loss: 1.2880 - learning_rate: 5.0000e-04
Epoch 7/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2199 - loss: 1.6134

KeyboardInterrupt: 